# Use Guardrails from LangChain

You can use Guardrails to add a layer of security around LangChain components. Here's how to use Guardrails with LangChain.

## Installing dependencies

Make sure you have both langchain and guardrails installed. If you don't, run the following commands:

In [ ]:
!pip install guardrails-ai
!pip install langchain

## Create a `RAIL` spec

In [2]:
rail_spec = """
<rail version="0.1">

<output>
    <object name="bank_run" format="length: 2">
        <string
            name="explanation"
            description="A paragraph about what a bank run is."
            format="length: 200 240"
            on-fail-length="noop"
        />
        <url
            name="follow_up_url"
            description="A web URL where I can read more about bank runs."
            required="true"
            format="valid-url"
            on-fail-valid-url="filter"
        />
    </object>
</output>

<prompt>
Explain what a bank run is in a tweet.

@xml_prefix_prompt

{output_schema}

@json_suffix_prompt_v2_wo_none
</prompt>
</rail>
"""

## Create a `GuardrailsOutputParser`

In [ ]:
from rich import print

from langchain.output_parsers import GuardrailsOutputParser

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [ ]:
output_parser = GuardrailsOutputParser.from_rail_string(rail_spec)

The `GuardrailsOutputParser` contains a `Guard` object, which can be used to access the prompt and output schema. E.g., here is the compiled prompt that is stored in `GuardrailsOutputParser`:

In [ ]:
print(output_parser.guard.base_prompt)

We can now create a LangChain `PromptTemplate` from this output parser.

## Create Prompt Template

In [ ]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
)

## Query the LLM and get formatted, validated and corrected output

In [ ]:
model = OpenAI(temperature=0)

output = model(prompt.format_prompt().to_string())

In [ ]:
output_parser.parse(output)